In [1]:
import os
import time
import pandas as pd
import numpy as np

import rasterio

import dask_gateway
import dask.array as da

from joblib import load

import raster_to_features as rm

In [2]:
#scene_ids = pd.read_csv(os.path.join(os.getcwd(),'temp','coastal_scenes_ids_2020.csv')).itemid.loc[19:26]
itemid = 'ca_m_3411933_se_11_060_20200521'

In [3]:
# open pre-trained random forest classifier
rfc = load('spectral_rfc.joblib') 
print('loaded model')

loaded model


In [6]:
# initialize DASK cluster
cluster = dask_gateway.GatewayCluster()
cluster.scale(30)

client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.10fecebbafc34a32a4585a67692e727a/status,


In [3]:
#for itemid in scene_ids:

# ---------------------------------------
# open NAIP scene
t_alpha = time.time()
raster = rm.rioxr_from_itemid(itemid)

# find vegetation pixels to go into model
# keep ndices of water and low-ndvi pixels
# adds ndvi and ndwi features for each pixel
t0 = time.time()
is_veg, water_index, not_veg_index = rm.add_spectral_features(df = rm.raster_as_df(raster.to_numpy(), 
                                                                                   ['r','g','b','nir']), #names of bands
                                                           ndwi_thresh = 0.3, 
                                                           ndvi_thresh = 0.05) 
# select features
is_veg.drop('ndwi', axis=1, inplace=True)
is_veg = rm.add_date_features(is_veg, raster.datetime)
print('assembled pixels dataframe with features\n   time taken to assemble: ', time.time() - t0,' s')


assembled pixels dataframe with features
   time taken to assemble:  15.175546884536743  s


In [4]:
is_veg

,r,g,b,nir,ndvi,year,month,day_in_year
0,126,113,92,166,0.136986,2020,5,142
1,129,119,92,169,0.134228,2020,5,142
2,118,110,88,158,0.144928,2020,5,142
3,111,107,84,156,0.168539,2020,5,142
4,107,110,83,167,0.218978,2020,5,142
...,...,...,...,...,...,...,...,...
132160511,22,38,50,34,0.214286,2020,5,142
132171088,20,35,46,23,0.069767,2020,5,142
132171090,22,38,47,33,0.200000,2020,5,142
132171091,21,37,48,29,0.160000,2020,5,142


In [5]:
np.array(is_veg)

array([[ 126.,  113.,   92., ..., 2020.,    5.,  142.],
       [ 129.,  119.,   92., ..., 2020.,    5.,  142.],
       [ 118.,  110.,   88., ..., 2020.,    5.,  142.],
       ...,
       [  22.,   38.,   47., ..., 2020.,    5.,  142.],
       [  21.,   37.,   48., ..., 2020.,    5.,  142.],
       [  19.,   35.,   46., ..., 2020.,    5.,  142.]])

In [7]:
# ---------------------------------------
# convert into dask.array and predict using model
da_pixels = da.from_array(np.array(is_veg), chunks=728802)
da_pixels

dask.array<array, shape=(19293593, 8), dtype=float64, chunksize=(728802, 8), chunktype=numpy.ndarray>

In [9]:
scene_preds = rfc.predict(da_pixels)    

In [5]:
    t0 = time.time()
    preds = scene_preds.compute()
    print('finished predicting\n   time taken to predict: ', time.time() - t0,' s')

In [5]:
    # ---------------------------------------
    # recover pixel indices for iceplant classifications
    preds_df = pd.DataFrame(preds, 
                         columns=['is_iceplant'], 
                         index = is_veg.index)
    is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
    non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

    # ---------------------------------------
    # reconstruct indices into image
    indices = [non_iceplant_index,
               is_iceplant_index, 
               not_veg_index,
               water_index]
    values = [0,    # values assigned to pixels from each index
              1,
              2,
              3]
    t0 = time.time()
    reconstruct = rm.indices_to_image(raster.shape[1], raster.shape[2], indices, values, back_value=100)
    print('reconstructed image\n   time taken to reconstruct: ', time.time() - t0,' s')

    # ---------------------------------------
    # save raster 
    filename = 'S_preds_' + itemid +'.tif'

    with rasterio.open(
        os.path.join(os.getcwd(),'temp',filename),  # file path
        'w',           # w = write
        driver = 'GTiff', # format
        height = reconstruct.shape[0], 
        width = reconstruct.shape[1],
        count = 1,  # number of raster bands in the dataset
        dtype = rasterio.uint8,
        crs = raster.rio.crs,
        transform = raster.rio.transform(),
    ) as dst:
        dst.write(reconstruct.astype(rasterio.uint8), 1)
    # ---------------------------------------
    print('total tme:', time.time() - t_alpha)
    print('FINISHED: ', itemid , '\n')